# Aplicação do algoritmo PCA

In [4]:
import numpy as np
import pandas as pd

Nosso trabalho aqui será reduzir a dimensionalidade de um dataset que contém as respostas de 40 alunos a um quiz com 49 perguntas. Cada resposta representa uma variável em nosso problema. Os dados são fictícios e de exemplo.

A primeira coluna (que carregaremos como índice) representa ID anônimo do aluno.

In [8]:
# Importando o dataset
df = pd.read_csv("/home/priscila/Downloads/1-Mat/dados/dataset_mat_p3.csv", index_col=0)
df.head()

,186,295,321,337,464,469,502,506,563,931,...,1401,1402,1414,1422,1524,1551,1553,1559,1561,1568
NSCLC_A549_1,171210.5829,1246686.524,527193.4870,411155.8781,59929.70461,49255.07973,171273.1364,159183.1286,663694.9741,35070.49713,...,95049.95991,1.719464e+06,645473.1920,89954.00002,52983.82193,1.672322e+06,6.867305e+05,259889.4715,1.563879e+06,189971.51110
NSCLC_H1703_2,204751.3591,1338013.461,571379.0841,486137.0920,89261.66256,72052.72202,187464.2389,180000.6140,443440.3745,89938.26439,...,131002.12740,8.290090e+05,293504.7109,42264.73772,97472.88420,1.608167e+06,9.408252e+05,363398.5823,3.043114e+05,35762.79339
NSCLC_H1703_1,203558.4952,1040438.105,498460.6875,411052.8868,96002.36973,0.00000,193894.4078,179518.9387,736672.9068,107041.34120,...,108827.84800,8.541730e+05,313811.6234,51125.12753,95633.56928,1.931491e+06,1.086556e+06,417606.0810,4.365568e+05,53865.65208
NSCLC_A549_2,245859.2006,1371135.588,153050.9373,495539.7034,81436.65785,85158.56741,0.0000,186757.1440,809830.2341,43565.82607,...,106003.59820,2.037000e+06,762600.1361,116878.29910,58497.52991,2.077164e+06,8.461167e+05,328548.1739,1.824857e+06,243842.05260
NSCLC_H1437_1,214448.1780,1107105.986,524333.5670,484994.8797,92368.82235,69868.54980,181168.8533,0.0000,635990.2172,51087.42828,...,83218.09256,1.269266e+06,459612.8875,64837.50435,0.00000,9.434720e+05,4.022300e+05,153070.9443,3.745915e+05,44104.41364


In [12]:
matriz = df.to_numpy()


### Versão 1

In [13]:
input = matriz

In [33]:
media = np.mean(input, 0)

input_normalizado = np.subtract(input, media)

input_normalizado_transposta = np.transpose(input_normalizado)

num_amostras = input.shape[0]

cov_mat = (np.dot(input_normalizado_transposta, input_normalizado)) / num_amostras 

valores, vetores = np.linalg.eig(cov_mat)

index = np.argsort(valores)[::-1]
valores = valores[index]
vetores = vetores[:, index]

pca_output = np.dot(input_normalizado, vetores)

variancias_acumuladas = np.cumsum(valores) / np.sum(valores)
variancias_acumuladas[:10]

array([0.400639  +0.j, 0.54972871+0.j, 0.67969581+0.j, 0.77504452+0.j,
       0.83944882+0.j, 0.87759586+0.j, 0.90642454+0.j, 0.92870154+0.j,
       0.94506171+0.j, 0.95783502+0.j])

Cerca de 96% da variância é explicada pelos primeiros 10 componentes principais.

### Versão 2 - mais otimizada

In [35]:
# Função com implementação do algoritmo PCA
def PCA_V2(input_data):
    
    # Calculando a média de cada coluna
    mean = np.mean(input_data, axis=0)
    
    # Subtraindo a média da matriz de entrada original
    normalised_input = input_data - mean
    
    # Calculando a matriz de covariância
    # A divisão é feita por (num_of_samples - 1) para a covariância da amostra
    cov_mat = np.cov(normalised_input, rowvar = False)
    
    # Localizando autovalores e autovetores usando uma função mais adequada para matrizes simétricas
    eigenvalues, eigenvectors = np.linalg.eigh(cov_mat)
    
    # Ordenando os autovetores de acordo com os autovalores em ordem decrescente
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    
    # Multiplicando a matriz de componente principal com a matriz de dados de entrada para obter o PCA
    pca_output = np.dot(normalised_input, sorted_eigenvectors)

    return pca_output, sorted_eigenvalues, sorted_eigenvectors, cov_mat

In [37]:
# Executando o PCA
pca_out, eigenval_v2, principal_component, cov_matrix = PCA_V2(matriz)

In [39]:
total_variance = np.sum(eigenval_v2)
explained_variances = [(i / total_variance) for i in eigenval_v2]
cumulative_explained_variance = np.cumsum(explained_variances)
cumulative_explained_variance[:10]

array([0.400639  , 0.54972871, 0.67969581, 0.77504452, 0.83944882,
       0.87759586, 0.90642454, 0.92870154, 0.94506171, 0.95783502])

### Versão 3

In [40]:
import sklearn
from sklearn.decomposition import PCA

In [43]:
pca = PCA(n_components=10)
X_pca = pca.fit_transform(matriz)

In [44]:
variancia_explicada = pca.explained_variance_ratio_
variancia_explicada_acumulada = np.cumsum(variancia_explicada)
variancia_explicada_acumulada[:10]

array([0.400639  , 0.54972871, 0.67969581, 0.77504452, 0.83944882,
       0.87759586, 0.90642454, 0.92870154, 0.94506171, 0.95783502])